# Getting Started

In this tutorial, we'll be looking at how to make a sentiment analysis model using Reccurent Neural Networks in Tensorflow. RNNs are the go-to for most NLP tasks today. The big advantage of the RNN is that it is able to effectively use data from previous time steps. The most distinct difference from a traditional NN is that an RNN takes in a sequence of inputs (words in our case). You can contrast this to a typical CNN where you’d have just a singular image as input. With an RNN, however, the input can be anywhere from a short sentence to a 5 paragraph essay. Additionally, the order of inputs in this sequence can largely affect how the weight matrices and hidden state vectors change during training. The hidden states, after training, will hopefully capture the information from the past (the previous time steps).

Let's look at how we can create an RNN model in Tensorflow that classifies sentences as having positive or negative sentimnent. 

In [1]:
import numpy as np
import tensorflow as tf
import collections
import tflearn

# Loading Our Dataset

For our sentiment analysis task, we'll be classifying movie reviews from IMDB. To get this dataset, we'll be using the TFlearn library

In [2]:
from tflearn.datasets import imdb
train, test, _ = imdb.load_data(path='imdb.pkl', n_words=10000,valid_portion=0.1)

In [3]:
trainX, trainY = train
testX, testY = test

## Obtaining Pretrained Word Vectors Through GloVe 

First, before we go into RNNs, we need to obtain some pretrained word vectors. The idea behind word vectors is that we want to represent each word as a d dimensional vector. Each of them will be 50 dimensional. We're going to store the words and the corresponding vector in a Python dictionary.

In [4]:
filename = 'glove.6B.50d.txt' #This text file has to be in the same folder. You can download it from 
                              #here: http://nlp.stanford.edu/projects/glove/ 
numWords = 400000
numDimensions = 50
fin=open(filename,'r')
wordList=[] #Just contains a list of all the words 
wordVectorMatrix = np.zeros((numWords + 1,numDimensions)) # numWords+1 x numDimensions matrix representing the word vector matrix
wordVectorDictionary = collections.defaultdict(list)
with open(filename) as inputfile:
    index = 0
    for line in inputfile:
        split = line.split()
        word = (split[0])
        wordList.append(word)
        listValues = split[1:]
        listValues = map(float, listValues)
        wordVectorMatrix[index] = listValues
        wordVectorDictionary[word] = listValues
        index += 1

# RNN Model

Let's first take a look at what our inputs and outputs are for this network. The input is going to be a sentence of max length 20 words, and the output will be a single number representing the sentiment of the sentence (1 for extremely positive and 0 for extremely negative)

In [5]:
batch_size = 1
numClasses = 2
max_seq_length = 10

In [12]:
tf.reset_default_graph()
data = tf.placeholder(tf.float32, [None, max_seq_length,numDimensions]) #Number of examples, number of input, dimension of each input
target = tf.placeholder(tf.float32, [None, numClasses])
num_hidden = 24
cell = tf.nn.rnn_cell.LSTMCell(num_hidden,state_is_tuple=True)
val, _ = tf.nn.dynamic_rnn(cell, data, dtype=tf.float32)
val = tf.transpose(val, [1, 0, 2])
last = tf.gather(val, int(val.get_shape()[0]) - 1)
weight = tf.Variable(tf.truncated_normal([num_hidden, int(target.get_shape()[1])]))
bias = tf.Variable(tf.constant(0.1, shape=[target.get_shape()[1]]))
prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
cross_entropy = -tf.reduce_sum(target * tf.log(tf.clip_by_value(prediction,1e-10,1.0)))
train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

In [18]:
# Our test sentence is going to be "I thought the movie was incredible and inspiring"
xTrain = np.zeros((1, max_seq_length, numDimensions)) # 10 x 50 matrix describing the input sentence
xTrain[0][0] = wordVectorDictionary["i"]
xTrain[0][1] = wordVectorDictionary["thought"]
xTrain[0][2] = wordVectorDictionary["the"]
xTrain[0][3] = wordVectorDictionary["movie"]
xTrain[0][4] = wordVectorDictionary["was"]
xTrain[0][5] = wordVectorDictionary["incredible"]
xTrain[0][6] = wordVectorDictionary["and"]
xTrain[0][7] = wordVectorDictionary["inspiring"]
#xTrain[8] and xTrain[9] are going to just be filled with zeros
print xTrain.shape
yTrain = np.zeros((1, numClasses))
yTrain[0] = 1
print yTrain.shape

(1, 10, 50)
(1, 2)


In [24]:
import datetime
tf.summary.scalar('CrossEntropy_Loss', cross_entropy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [25]:
init_op = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init_op)

epoch = 50
for i in range(epoch):
    summary = sess.run(merged,{data: xTrain, target: yTrain})
    writer.add_summary(summary, i)
    train_step.run(session=sess,feed_dict={data: xTrain, target: yTrain})
    print "Epoch ",str(i)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch  0
Epoch  1
Epoch  2
Epoch  3
Epoch  4
Epoch  5
Epoch  6
Epoch  7
Epoch  8
Epoch  9
Epoch  10
Epoch  11
Epoch  12
Epoch  13
Epoch  14
Epoch  15
Epoch  16
Epoch  17
Epoch  18
Epoch  19
Epoch  20
Epoch  21
Epoch  22
Epoch  23
Epoch  24
Epoch  25
Epoch  26
Epoch  27
Epoch  28
Epoch  29
Epoch  30
Epoch  31
Epoch  32
Epoch  33
Epoch  34
Epoch  35
Epoch  36
Epoch  37
Epoch  38
Epoch  39
Epoch  40
Epoch  41
Epoch  42
Epoch  43
Epoch  44
Epoch  45
Epoch  46
Epoch  47
Epoch  48
Epoch  49
